In [5]:
import pandas as pd
import json
import os

In [6]:
def load_data(data_path):
    """Loads all data files from the specified path."""
    try:
        performance_df = pd.read_csv(os.path.join("/kaggle/input/linkedin-assignment/performance_metrics.csv"))
        inventory_df = pd.read_csv(os.path.join("/kaggle/input/linkedin-assignment/inventory_movements.csv"))
        catalog_df = pd.read_csv(os.path.join("/kaggle/input/linkedin-assignment/internal_catalog_dump.csv"))
        
        with open(os.path.join("/kaggle/input/linkedin-assignment/marketplace_snapshot.json"), 'r') as f:
            marketplace_data = json.load(f)
            
        with open(os.path.join("/kaggle/input/linkedin-assignment/competitor_intelligence.json"), 'r') as f:
            competitor_data = json.load(f)
            
        print("All data files loaded successfully.")
        return performance_df, inventory_df, catalog_df, marketplace_data, competitor_data
    except FileNotFoundError as e:
        print(f"Error loading data: {e}. Make sure all input files are in the correct directory.")
        return None

In [7]:
def analyze_profitability_and_performance(perf_df, catalog_df):
    """Analyzes ad performance, flagging low ROAS and negative profitability."""
    print("Analyzing profitability and performance...")
    insights = []
    
    merged_df = pd.merge(perf_df, catalog_df, left_on='identifier', right_on='item_code', how='left')
    
    merged_df['roas'] = merged_df['revenue'] / merged_df['ad_spend']
    merged_df['cogs'] = merged_df['conversions'] * merged_df['cost_basis']
    merged_df['profit_after_ad_spend'] = merged_df['revenue'] - merged_df['ad_spend'] - merged_df['cogs']
    
    roas_threshold = 2.0 
    
    for _, row in merged_df.iterrows():
        if row['roas'] < roas_threshold:
            insights.append({
                "priority": 2,
                "type": "Low ROAS",
                "sku": row['identifier'],
                "channel": row['channel'],
                "details": f"ROAS is {row['roas']:.2f} for week ending {row['week_ending']}, which is below the target threshold of {roas_threshold}. This indicates inefficient ad spend."
            })
            

        if row['profit_after_ad_spend'] < 0:
            insights.append({
                "priority": 1,
                "type": "Negative Profitability",
                "sku": row['identifier'],
                "channel": row['channel'],
                "details": f"Product generated a negative profit of ${row['profit_after_ad_spend']:.2f} after ad spend and COGS for week ending {row['week_ending']}. Ad strategy is unsustainable."
            })
            
    return insights

In [8]:
def analyze_availability(marketplace_data, perf_df):
    """Analyzes product availability, flagging OOS and low stock issues."""
    print("Analyzing product availability...")
    insights = []
    

    amazon_products = marketplace_data.get('platforms', {}).get('amazon', {}).get('products', [])
    sku_map = {
        "B09K4RT5MN": "NT-EARBUD-X1", "B09K4RT6NP": "NT-EARBUD-X1-BLK",
        "B08M3PLK7Y": "NT-SPEAKER-Z2", "B0B7NK4L9M": "NT-CHARGER-Q3",
        "B09Q8RT3KL": "NT-WATCH-S5", "B0C1M5RT8K": "NT-TABLET-P6"
    }

    for product in amazon_products:
        sku = sku_map.get(product['asin'])
        if not sku:
            continue
            

        if product.get('availability') == 'temporarily_unavailable' or product.get('estimated_stock', 1) == 0:

            recent_spend = perf_df[(perf_df['identifier'] == sku) & (perf_df['channel'] == 'Amazon')]['ad_spend'].sum()
            if recent_spend > 0:
                insights.append({
                    "priority": 1,
                    "type": "Wasted Spend - Out of Stock",
                    "sku": sku,
                    "channel": "Amazon",
                    "details": f"Product is Out of Stock (OOS) but had recent ad spend of ${recent_spend:,.2f}, leading to wasted clicks and revenue loss."
                })
        

        elif product.get('availability') == 'low_stock' or (product.get('estimated_stock') is not None and product.get('estimated_stock') < 20):
             insights.append({
                "priority": 2,
                "type": "Low Stock Risk",
                "sku": sku,
                "channel": "Amazon",
                "details": f"Inventory is critically low with an estimated {product.get('estimated_stock')} units. High risk of going OOS, impacting sales velocity and ad campaign performance."
            })
            
    return insights

In [9]:
def analyze_digital_shelf(marketplace_data, catalog_df):
    """Analyzes pricing, buy box, content, and reviews."""
    print("Analyzing digital shelf (Pricing, Buy Box, Content)...")
    insights = []
    
    amazon_products = marketplace_data.get('platforms', {}).get('amazon', {}).get('products', [])
    sku_map = {
        "B09K4RT5MN": "NT-EARBUD-X1", "B09K4RT6NP": "NT-EARBUD-X1-BLK",
        "B08M3PLK7Y": "NT-SPEAKER-Z2", "B0B7NK4L9M": "NT-CHARGER-Q3",
        "B09Q8RT3KL": "NT-WATCH-S5", "B0C1M5RT8K": "NT-TABLET-P6"
    }

    for product in amazon_products:
        sku = sku_map.get(product.get('asin'))
        if not sku:
            continue
        
        catalog_info = catalog_df[catalog_df['item_code'] == sku].iloc[0] if not catalog_df[catalog_df['item_code'] == sku].empty else None

        # 1. Buy Box Analysis
        if product.get('buy_box_winner') and product.get('buy_box_winner') != 'NovaTech Official':
            insights.append({
                "priority": 1,
                "type": "Lost Buy Box",
                "sku": sku,
                "channel": "Amazon",
                "details": f"Losing the Buy Box to '{product.get('buy_box_winner')}'. This directly diverts sales to a third-party seller."
            })
            

        if catalog_info is not None and 'min_advertised_price' in catalog_info:
            map_price = catalog_info['min_advertised_price']
            current_price = product.get('current_price')
            if current_price and current_price < map_price:
                insights.append({
                    "priority": 2,
                    "type": "MAP Violation",
                    "sku": sku,
                    "channel": "Amazon",
                    "details": f"Product is sold at ${current_price}, which is below the MAP of ${map_price}. This erodes brand value and partner trust."
                })


        if product.get('content_quality_flags'):
            insights.append({
                "priority": 3,
                "type": "Content Opportunity",
                "sku": sku,
                "channel": "Amazon",
                "details": f"Listing has content issues: {', '.join(product.get('content_quality_flags'))}. Improving content can increase conversion rate."
            })


        if product.get('avg_rating') and product.get('avg_rating') < 4.0:
            insights.append({
                "priority": 2,
                "type": "Poor Reviews",
                "sku": sku,
                "channel": "Amazon",
                "details": f"Average rating is {product.get('avg_rating')}/5 based on {product.get('review_count')} reviews. This can deter potential buyers and suppress CVR."
            })
            
    return insights

In [10]:
def main():
    """Main function to run the diagnostic analysis."""


    data_path = os.path.join("data_path = '/kaggle/input/novatech-inputs'")
    output_path = os.path.join("/kaggle/working/")
    

    if not os.path.exists(output_path):
        os.makedirs(output_path)

    loaded_data = load_data(data_path)
    if not loaded_data:
        return
    performance_df, inventory_df, catalog_df, marketplace_data, competitor_data = loaded_data
    

    all_insights = []
    all_insights.extend(analyze_profitability_and_performance(performance_df, catalog_df))
    all_insights.extend(analyze_availability(marketplace_data, performance_df))
    all_insights.extend(analyze_digital_shelf(marketplace_data, catalog_df))
    

    sorted_insights = sorted(all_insights, key=lambda x: x['priority'])
    

    output_file = os.path.join(output_path, 'diagnostic_report.json')
    with open(output_file, 'w') as f:
        json.dump(sorted_insights, f, indent=4)
        
    print(f"\nAnalysis complete. Diagnostic report saved to: {output_file}")
    print(f"Total issues/opportunities identified: {len(sorted_insights)}")

In [11]:
if __name__ == '__main__':
    main()

All data files loaded successfully.
Analyzing profitability and performance...
Analyzing product availability...
Analyzing digital shelf (Pricing, Buy Box, Content)...

Analysis complete. Diagnostic report saved to: /kaggle/working/diagnostic_report.json
Total issues/opportunities identified: 9
